## Data augmentation for ablation test

Use GPT for data augmentation, 为了通用化测试一个模型的六种能力，Creativity, Logical Reasoning, In-context learning, Instruciton following, Machine Translation 以及Summarization.
你需要针对一个模型的多种能力同时进行循序渐进的测试，我给你分别六种能力的一个问题的示例，请模仿类似的问题给我一个新的问题，并给我反馈结果或者答案。 

In [2]:
pip install openai

27.46s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [4]:
import openai
import pandas as pd
import random
import os
import json

# ChatGPT API 配置：替换为你的 API 密钥
openai.api_key = "sk-pA"

# 定义能力字典
ability_dict = {
    "CR": "Causal Reasoning",
    "CT": "Creativity",
    "ICL": "In-context Learning",
    "IF": "Instruction Following",
    "MT": "Machine Translation",
    "SUMM": "Summarization",
}

# 灵感生成模板
inspiration_prompt_template = """
I want you to take on the role of an ingenious and imaginative assistant. Your mission is to spark creativity and inspire innovative thinking. For the given topic of {capability} ability, please provide a truly unique and thought-provoking inspiration sentence. 

This inspiration should:
1. Challenge conventional thinking and encourage thinking outside the box.
2. Stimulate a wide array of creative and critical ideas.
3. Serve as a catalyst for generating engaging and meaningful tasks or questions.

Be bold, imaginative, and daring in your approach. Let's create something extraordinary!
"""

# 问题生成模板
prompt_template = """
The goal of this task is to evaluate the model's ability to demonstrate {capability}. Imagine you are designing a challenging yet creative evaluation question. Using the example provided below as a guide and the inspiration as a springboard, craft a new and exciting question that captures the essence of this capability. 

Make the question engaging, thought-provoking, and appropriately complex, while ensuring it evaluates the desired skill effectively. Generate a JSON object with the following fields: question (string), reference (string).


Here are the inputs:
- Example: {example}
- Inspiration: {inspiration}

Let your imagination flow and create a question that stands out!
"""

# 创建本地输出目录
output_dir = "/home/lujun_li/projects/temperature_eval/data/Augemented/"
os.makedirs(output_dir, exist_ok=True)

# 为每种能力生成问题
for ability_code, capability_name in ability_dict.items():
    try:
        # 加载能力对应的 CSV 文件
        target_df = pd.read_csv(
            f"/home/lujun_li/projects/temperature_eval/data/Intermediate/{ability_code}.csv"
        )

    except (FileNotFoundError, KeyError):
        print(
            f"Warning: File for {ability_code} not found or invalid format, skipping."
        )
        continue

    # 存储生成数据
    generated_data = []

    # 执行 50 次随机抽样（有放回抽样）
    if ability_code == "MT":
        repetition = 100
    if ability_code == "CT":
        repetition = 10
    else:
        repetition = 50
    for _ in repetition(repetition):
        # 随机选择一行（有放回采样）
        random_row = target_df.sample(n=1, random_state=random.randint(0, 100)).iloc[0]
        example = random_row["input"]  # 如果没有 "example" 列，返回空字符串

        # 调用模型生成灵感
        inspiration_prompt = inspiration_prompt_template.format(
            capability=capability_name
        )
        inspiration_response = openai.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": inspiration_prompt},
            ],
            temperature=0.7,  # 调整随机性
        )
        inspiration = inspiration_response.choices[0].message.content.strip()

        # 调用模型生成问题
        prompt = prompt_template.format(
            capability=capability_name,
            example=example,
            inspiration=inspiration,
        )
        question_response = openai.chat.completions.create(
            model="gpt-4",
            messages=[
                {
                    "role": "system",
                    "content": "You are an assistant that only outputs JSON-formatted data. No extra explanations, no text outside JSON, and strict adherence to the required format.",
                },
                {"role": "user", "content": prompt},
            ],
            temperature=0.7,  # 调整随机性
        )
        question_json = question_response.choices[0].message.content.strip()

        # 解析 JSON 输出
        try:
            question_json = json.loads(question_json)  # 尝试解析 JSON
            question = question_json.get("question", "")
            reference = question_json.get("reference", "")
        except json.JSONDecodeError:
            print(f"Invalid JSON format: {question_json}")
            question = ""
            reference = ""

        # 保存生成结果到数据结构中
        generated_data.append(
            {
                "Ability": capability_name,
                "Example": example,
                "Inspiration": inspiration,
                "Question": question,
                "Reference": reference,
            }
        )

        # 打印生成结果
        print(f"Ability: {capability_name}")
        print(f"Inspiration: {inspiration}")
        print(f"Generated Question: {question_json}")
        print("-" * 50)

    # 将生成的问题保存到 CSV 文件
    output_file = os.path.join(output_dir, f"{ability_code}_generated_questions.csv")
    pd.DataFrame(generated_data).to_csv(output_file, index=False, encoding="utf-8")
    print(f"Saved 50 questions for {capability_name} to {output_file}.")

Ability: Causal Reasoning
Inspiration: "Imagine a world where cause and effect are not linear, but rather a vibrant, interconnected web of possibilities - what new dimensions would it add to our understanding of causal reasoning, and how would it alter our approach to problem-solving?"
Generated Question: {'question': 'Please reference the following three examples (including premise, query, possible answers, and correct answer) to answer the final query. Premise: A ball falls into a pool. Query: What would have happened if a cube of sugar had fallen into the pool? Possible answers: The sugar cube would have remained intact. The sugar cube would have dissolved. That is not possible. Correct answer: The sugar cube would have dissolved. Premise: A candle burns. Query: What would have happened if a stone was put to the flame? Possible answers: The stone would have melted. Nothing would have changed. The stone would have burned. Correct answer: Nothing would have changed. Premise: A woman r